# 在命名空间中处理Tcl脚本

## namespace eval

作用：在指定的命名空间中处理Tcl脚本，并返回脚本结果。若指定的命名空间不存在，则创建该命名空间。**注意，若命令在指定的命名空间中找不到，则在全局命名空间中找。**

In [20]:
proc whoAmI {} {
    return "global command"
}
namespace eval ns {
    proc whoAmI {} {
        return "namespace command"
    }
}

In [5]:
whoAmI

global command

In [6]:
ns::whoAmI

namespace command

In [21]:
namespace eval ns {
    whoAmI
}

namespace command

## namespace delete

作用：删除命名空间。

In [22]:
# 接上节
puts [ns::whoAmI]
namespace delete ns
puts [ns::whoAmI]

namespace command


invalid command name "ns::whoAmI"

## variable

作用：设置或访问命名空间中的变量。

In [25]:
namespace eval counter {
    # 在counter命名空间中定义一个变量
    variable num 10
    
    proc next {} {
        # 使得counter命名空间中的num变量在proc过程中可访问
        variable num
        return [incr num]
    }
    
    proc reset {} {
        variable num
        set num 0
    }
}

In [26]:
counter::next

11

In [27]:
counter::next

12

In [28]:
counter::reset

0

In [30]:
counter::next

1

**有趣的发现：**
命名空间的过程中创建的变量在全命名空间都能访问的到。

In [1]:
namespace eval counter {
    proc next {} {
        variable num
        return [incr num]
    }
    
    proc reset {} {
        variable num
        set num 0
    }
}

In [2]:
puts [counter::next]
puts [counter::next]
puts [counter::next]
puts [counter::next]

puts [counter::reset]

puts [counter::next]
puts [counter::next]
puts [counter::next]
puts [counter::next]

1
2
3
4
0
1
2
3
4


**variable不能初始化数组的值，但可以使过程访问数组。即过程中通过variable创建的变量具有全局意义。**

In [1]:
namespace eval catalog {
    variable entries
    array set entries {}
    
    proc add {item} {
        variable entries
        incr entries($item)
    }
    
    proc getEntries {} {
        variable entries
        return [lsort [array names entries]]
    }
    
    proc countInstances {item} {
        variable entries
        return $entries($item)
    }
}
puts [catalog::add apple]
puts [catalog::add orange]
puts [catalog::add apple]
puts [catalog::add banana]
puts [catalog::getEntries]
puts [catalog::countInstances apple]

1
1
2
1
apple banana orange
2


In [1]:
namespace eval catalog {
    #variable entries
    array set entries {}
    
    proc add {item} {
        variable entries
        incr entries($item)
    }
    
    proc getEntries {} {
        variable entries
        return [lsort [array names entries]]
    }
    
    proc countInstances {item} {
        variable entries
        return $entries($item)
    }
}
puts [catalog::add apple]
puts [catalog::add orange]
puts [catalog::add apple]
puts [catalog::add banana]
puts [catalog::getEntries]
puts [catalog::countInstances apple]

1
1
2
1
apple banana orange
2


## namespace code

作用：生成处理回调的脚本。如组件的-command选项，after，trace命令，lsort的-command选项。该命令会把与脚本相关的代码打包在一起，确保它是在当前命名空间中处理的，而不是在它的调用者的命名空间中处理。

In [3]:
namespace eval catalog {
    proc save {file} {
        variable entries
        set f [open $file w]
        puts $f [list array set ::catalog::entries] [array get entries]
        close $f
    }
    
    proc autocommit {interval file} {
        after $interval [namespace code [list autocommit $interval $file]]
        save $file
    }
    
    # Save to catalogDB.tcl every 10 seconds
    autocommit 10000 catalogDB.tcl
}

invalid command name "file58"

# 操作限定名称

**限定名称**：绝对限定名称(::alpha::beta::gamma)和相对限定名称(alpha::beta)。


## namespace qualifiers

作用：获取限定名称的路径(头)

## namespace tail

作用：获取限定名称的尾

In [7]:
puts [namespace qualifiers alpha::beta]
puts [namespace qualifiers ::alpha::beta::gamma]
puts [namespace qualifiers delta]
puts [namespace tail alpha::beta]
puts [namespace tail ::alpha::beta::gamma]
puts [namespace tail delta]

alpha
::alpha::beta

beta
gamma
delta


***

**限定名称中使用变量名替换**：

In [8]:
set theNS ::alpha::beta
set theCommand $theNS::gamma

can't read "theNS::gamma": no such variable

In [9]:
set theCommand ${theNS}::gamma

::alpha::beta::gamma

# 在命名空间中导出和导入

## namespace export与namespace import

In [2]:
namespace eval src {
    proc a {} {return "alpha"}
    proc b {} {return "beta"}
    proc etc {} {return "..."}
    namespace export a b
}
namespace eval dst {
    namespace import ::src::*
    proc c {} {return "cahrlie"}
    expr {"[a] [b] [c]"}
}

alpha beta cahrlie

In [3]:
namespace eval src {
    proc d {} {return "gamma"}
}
expr {"[src::d] [src::a] [src::b]"}

gamma alpha beta

***

In [1]:
namespace eval src {
    proc a {} {return "src->a"}
    proc b {} {return "src->b"}
    namespace export a b
}
namespace eval dst {
    proc a {} {return "dst->a"}
    namespace import ::src::*
    expr {"[a] [b]"}
}

can't import command "a": already exists

In [3]:
namespace eval src1 {
    proc a {} {return "src1->a"}
    proc b {} {return "src1->b"}
    namespace export a b
}
namespace eval src2 {
    proc a {} {return "src2->a"}
    proc b {} {return "src2->b"}
    namespace export a b
}
namespace eval dst {
    namespace import ::src1::*
    namespace import ::src2::*
    expr {"[::src1::a]"}
    #expr {"[a] [b]"}
}

can't import command "a": already exists

<font color='#dd4b2a' size=3>同一个命名空间中不能出现相同名称的变量或命令，即使加上限定也不可以。</font>

In [1]:
namespace eval src1 {
    proc a {} {return "src1->a"}
    proc b {} {return "src1->b"}
    namespace export a b
}
namespace eval src2 {
    proc a {} {return "src2->a"}
    proc b {} {return "src2->b"}
    namespace export a b
}
namespace eval dst {
    namespace import ::src1::*
    namespace import -force ::src2::*
    expr {"[a] [b]"}
}

src2->a src2->b

<font color='#f40c7a' size=3>namespace import -force强制覆盖先前导入的变量和名称。</font>

# 检查命名空间

* namespace current：获得当前命令空间的名称
* namespace parent：获得当前命名空间的父命名空间的名称
* namespace children：获得当前命名空间的子命名空间的名称(可接受通配符模式参数)

In [2]:
namespace eval example {
    namespace current
}

::example

In [4]:
namespace eval example {
    namespace parent
}

::

In [5]:
namespace eval example {
    namespace eval abc1 {}
    namespace eval abc2 {}
    namespace eval abc3 {}
}
namespace children example abc*

::example::abc2 ::example::abc3 ::example::abc1

***

* info command：查看命名空间中的命令和变量
* namespace which：查看一个命令或变量的非限定名称的完全限定名称

In [14]:
namespace eval example {
    proc eg1 {} {}
    proc eg2 {} {}
    set var 0
    variable var1 0
}
info command example::*

::example::eg ::example::eg1 ::example::env ::example::eg2

In [11]:
puts [namespace which eg1]

In [17]:
namespace eval example {
    proc eg {} {}
    proc env {} {}
    puts [namespace which eg] 
    puts [namespace which set] 
    puts [namespace which -variable env]
    puts [namespace which env]
}

::example::eg
::set
::env
::example::env


****

* namespace origin：查看一个可被导入的命令是由哪个命名空间导出的。

In [18]:
namespace eval example {
    proc eg1 {} {}
    proc eg2 {} {}
    namespace export *
}
namespace import example::*
namespace origin eg1

::example::eg1

In [19]:
rename eg2 example2
namespace origin example2

::example::eg2

namespace origin可查看命令的原始来源，即使它被重命名过。

# 有关集合命令

>常见的像string，namespace等都是一种集合形式。

## 基本的集合命令

* namespace ensemble create：创建集合命令(通常与namespace export一起用，将export的部分导出为一个集合命令)
* namespace ensemble exists：测试一个命令是否为集合命令

In [3]:
namespace eval example {
    proc add {x y} {expr {$x + $y} }
    proc multiply {x y} {expr {$x * $y} }
    proc  minus {x y} {expr {$x - $y} }
    namespace export add multiply minus
    namespace ensemble create
}

::example

In [4]:
example add 2 3

5

In [5]:
example multiply 6.0 7

42.0

In [6]:
namespace ensemble exists example

1

In [7]:
namespace ensemble exists example::add

0

***

命名空间中的子命令在应用的时候可以缩写，只要后面是独一无二的。

In [9]:
example mul 3.1 1.3

4.03

## 在集合命令中设置集合命令

compute命令有两个子命令，分别计算形状的面积和实体体积。对于不同形状和不同实体，这两个子命令又各有其子命令。

In [18]:
namespace eval compute {
    variable pi 3.1415927
    namespace export *
    namespace ensemble create
    namespace eval area {
        namespace export *
        namespace ensemble create
    }
    namespace eval volume {
        namespace export *
        namespace ensemble create
    }
}
proc compute::area::circle {radius} {
    variable ::compute::pi
    return [expr {$pi * $radius * 2}]
}
proc compute::area::square {side} {
    return [expr {$side ** 2}]
}
proc compute::volume::sphere {radius} {
    variable ::compute::pi
    return [expr {4 * $pi / 3 * $radius ** 3}]
}
proc compute::volume::cube {edge} {
    return [expr {$edge ** 3}]
}

In [12]:
compute ?

unknown or ambiguous subcommand "?": must be area, or volume

In [13]:
compute area ?

unknown or ambiguous subcommand "?": must be circle, or square

In [14]:
compute area circle 4

25.1327416

In [19]:
compute vol sphere 2

33.51032213333333

## 控制集合命令的设置

1. namespace ensemble create option<br>
2. namespace ensemble configure option：修订1中选项的值<br>
3. option：<br>
-map：包含一个字典，用key代替value，类似重命名。<br>
-parameters：为-command指定的集合命令名传递参数，另一作用是命令错误时可以产生参数提示信息。<br>
-prefixes：是否启用子命令的简写模式<br>
-subcommands：明确定义子命令列表。如果定义的集合命令的子命令并不是该命名空间所有公开导出的命令的集合时，这一功能很有用。<br>
-unknown：<br>
-namespace：集合绑定到的命名空间是可读的，不过这个选项不能修改，它在创建时自动确定。<br>

In [20]:
proc raisePower {power value} {
    return "$value ^ $power = [expr {$value ** $power}]"
}
namespace ensemble create -command power -map {
    square {raisePower 2}
    cube {raisePower 3}
    sqroot {raisePower 0.5}
    invert {raisePower -1}
}

::power

In [21]:
power cube 4

4 ^ 3 = 64

In [22]:
power sqroot 49

49 ^ 0.5 = 7.0

In [23]:
power inv 0.0625

0.0625 ^ -1 = 16.0

***

In [3]:
proc raisePower {power value} {
    return "$value ^ $power = [expr {$value ** $power}]"
}
namespace ensemble create -command power -prefixes 0 -map {
    square {raisePower 2}
    cube {raisePower 3}
    sqroot {raisePower 0.5}
    invert {raisePower -1}
}

::power

In [4]:
power inv 0.0625

unknown subcommand "inv": must be cube, invert, sqroot, or square

***

In [13]:
proc raisePower {power value} {
    return "$value ^ $power = [expr {$value ** $power}]"
}
namespace ensemble create -command power -prefixes 0 -map {
    square {raisePower 2}
    cube {raisePower 3}
    sqroot {raisePower 0.5}
    invert {raisePower -1}
} -subcommands {
    square
    cube
}

::power

In [2]:
power square 2

2 ^ 2 = 4

In [14]:
power cube 2

2 ^ 3 = 8

In [8]:
power sqroot 4

unknown subcommand "sqroot": must be cube, invert, or square

In [15]:
power invert 0.5

unknown subcommand "invert": must be cube, or square

***

In [17]:
namespace eval t {
    proc a {} {return "alpha"}
    proc b {} {return "beta"}
    namespace export *
    namespace ensemble create -command test -subcommands {
        b
    }
}

::t::test

In [19]:
::t::test b

beta

In [20]:
::t::test a

unknown or ambiguous subcommand "a": must be b

***

In [87]:
proc raisePower {power value} {
    return "$value ^ $power = [expr {$value ** $power}]"
}
namespace ensemble create -command power -map {
    square {raisePower }
    cube {raisePower 3}
    sqroot {raisePower 0.5}
    invert {raisePower -1}
} -parameters {
    arg1
}

::power

In [84]:
power

wrong # args: should be "power 
    arg1
    arg2
 subcommand ?arg ...?"

In [88]:
power 2 square 3

3 ^ 2 = 9

***

In [79]:
proc a {} {return "new third"}
proc b {} {return "beta"}
namespace ensemble create -command origin -map {
    first {a}
    second {b}
}

proc fun {arg1 arg2} {
    puts "function is run!"
    puts "$arg1, $arg2"
    return " new third"
}
proc c {} {return "char"}
namespace ensemble create -command new -map {
    third {c}
} -unknown fun

::new

In [80]:
new hello

function is run!
::new, hello
char

In [1]:
rename string strCore
proc strReverse {str} {
    set result {}
    for {set i [strCore length $str]} {$i > 0} {} {
        incr i -1
        append result [strCore index $str $i]
    }
    return $result
}

proc unknownStrCmd {str subcommand args} {
    puts "$str, $subcommand, $args"
    set arg1 [lindex $args 0]
    set arg2 [lindex $args 1]
    puts "arg1=$arg1"
    puts "arg2=$arg2"
    puts result=[strCore $subcommand $arg1 $arg2]
    return " strCore $subcommand $arg1 $arg2"
}
namespace ensemble create -command string -map {
    reverse {strReverse}
    repeat {strCore repeat}
    replace {strCore replace}
} -unknown unknownStrCmd

::string

In [2]:
set str "hello"
set r [string index $str 2]
puts reuslt==$r

::string, index, hello 2
arg1=hello
arg2=2
result=l


wrong # args: should be "::tcl::string::index string charIndex"

## 访问其他命名空间的变量

* 使用该变量的完整限定名称。
* 可以把该变量的完全限定名称提供给global，upvar，variable，将变量导入当前命名空间。
* 使用`namespace upvar`将一个命名空间中的一组变量导入当前命名空间中。

In [3]:
proc blob {name a b c} {
    namespace eval $name {}
    namespace upvar $name a va b vb c vc
    set va $a
    set vb $b
    set vc $c
    namespace ensemble create -map [list \
        set [list ::blobSet $name] \
        sum [list ::blobSum $name] \
        end [list ::blobEnd $name] \
    ] -command $name
}
proc blobSet {ns var value} {
    namespace upvar $ns $var v
    set v $value
    return
}
proc blobSum {ns} {
    namespace upvar $ns a va b vb c vc
    return [expr {$va+$vb+$vc}]
}
proc blobEnd {ns} {
    rename $ns {}
    namespace delete $ns
}


In [4]:
blob example 1 2 3

::example

In [6]:
example set a 4
example sum

9

In [7]:
example set b 5
example sum

12

In [8]:
example end